In [26]:
import numpy as np
import scipy as sp

In [27]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [28]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [29]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation
        # Refer to this source for derivation: https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)
        # this represents the upward thrust coefficient, will be multiplied by the speed of the rotor squared, to get upward force
        # in Project Session 3 slide 23, this is F = kT * ω^2

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)
        # this represents the torque coefficient, will be multiplied by the speed of the rotor squared to get moment at each rotor
        # in Project Session 3 slide 23, this is M = kM * ω^2
        
# Simply treat as a global
p = Params()

### Writing a controller for our drone
In this notebook, we'll start writing a feedback controller for our model! Again, I will be leaving you guys alone a little bit more.

#### Recommended steps
- Copy your 3D quadrotor model from the previous lecture
- Work your way down in the `Controller` class
    - I implemented a basic structure, but feel free to diverge from this if you like!
    - Write the inverse mixing matrix formulation to from **u** to rotor rates
    - As discussed, the first week I expect you to just set up control in *z* to warm up!

### The dynamics

In [30]:
def whatami(x) -> str:
        """This function prints some basic info about a numpy ndarray for troubleshooting purposes"""
        return 'is a ' + str(x.__class__.__name__) + ' with shape ' + str(x.shape) + ' = ' + str(x)

class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:
        # NOTE(roy): I advise you to scroll through dynamics.py and see what the __init__
        # function in the base class does!
        super().__init__()


    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray) -> np.ndarray:
        """Accepts an array of rotor rates and returns an array of thrusts [F1, F2, F3, F4]"""
        # NOTE(roy): Implement the simple rotor trust model we discussed in the slides
        # To see how complex this can get, see for example: https://arxiv.org/pdf/1601.00733.pdf
        # force of each rotor is related to rotor rates squared times constant p.k_thrust
        thrusts =  rotor_rates * rotor_rates * p.k_thrust     
        return thrusts


    def step(self, t: float, input: QuadrotorCommands) -> QuadrotorState:
        F1, F2, F3, F4 = FullQuadrotorDynamics.rotor_thrust_model(input.rotor_rates)
        L = p.arm_length
        m_t = p.k_torque / p.k_thrust     # multiply the forces by this to get the torques

        mixing_matrix = np.array([
            [  1,      1,      1,        1],
            [  0,      L,      0,      - L],
            [- L,      0,      L,        0],
            [m_t,  - m_t,    m_t,    - m_t]
        ])

        u = mixing_matrix @ np.array([F1, F2, F3, F4])         # thrust and torques = mixing_matrix @ rotor_thrusts
        # if t == 0: print("u", whatami(u))                      # u is a ndarray with shape (4,) - a horizontal vector [thrust, MomentX, MomentY, MomentZ]

        u1 = u[0]                                               # thrust is first element
        # if t == 0: print("u1", whatami(u1))                    # u1 is a numpy.float64 with shape () = 10.237126413876851

        u2 = u[1:4]                                             # torques (aka moments) are the last three elements
        # if t == 0: print("u2", whatami(u2))                    # u2 is a ndarray with shape (3,) = [ 1.38783753e-19 -1.38783753e-19  0.00000000e+00]

         # NOTE(roy) this local function is a wrapper around the class state_derivative function
        # all this does is convert the our state dataclass from/into a vector that the scipy
        # integration solver can work with. See dynamics.py for the definition of these
        # from/to state vector functions.
        def state_derivative_wrapped(t: float, state: np.ndarray) -> np.ndarray:
            return FullQuadrotorDynamics.state_derivative(
                t,
                QuadrotorState.from_state_vector(state),
                u1,
                u2
            ).to_state_vector()
           
        state_vector = self.state.to_state_vector()
        # if t == 0: print('start state_vector =', state_vector)  # state_vector = [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
        # if t == 0: print('start self.state =', self.state)      # self.state = QuadrotorState(position=array([0., 0., 0.]), orientation=<Rot3 [0.0, 0.0, 0.0, 1.0]>, velocity=array([0, 0, 0]), angular_velocity=array([0., 0., 0.]))

        solution = sp.integrate.solve_ivp(state_derivative_wrapped, (0, self.dt), state_vector)
        self.state = QuadrotorState.from_state_vector(solution['y'][:, -1])
        # if t > 3.99: print('at time', t, 'state_vector =', state_vector)  # state_vector = [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
        # if t > 3.99: print('at time', t, 'self.state =', self.state)      # self.state = QuadrotorState(position=array([0., 0., 0.]), orientation=<Rot3 [0.0, 0.0, 0.0, 1.0]>, velocity=array([0, 0, 0]), angular_velocity=array([0., 0., 0.]))
        return self.state
    
    
    @staticmethod
    def state_derivative(t: float, state: QuadrotorState, u1: np.ndarray, u2:np.ndarray) -> QuadrotorState:
        R = state.orientation           # a Rot3 (SymForce quaternion) such as <Rot3 [1.0, 3.0, 2.0, 1.0]>
        w = state.angular_velocity      # (3,) horizontal vector
        I = p.inertia                   # (3, 3) matrix

        # acceleration = downward gravity + (thrust rotated from body-frame to world-frame) /  mass
        accel = (np.array([0, 0, -  p.g]) + R * np.array([0, 0, u1])) / p.mass       # (3,) = (3,) + Rot3 * (3,)
        
        # rotate the torques from body-frame to world-frame
        M = (R * u2)                # (3,) = Rot3 * (3,)        (horizontal vector) = quaternion * (horizontal vector)

        # this is from project session 3, page 38: omega_dot = I_inverse @ (M - omega CrossProduct Iomega)  (do not use dotproduct for Roy's "dot" in the formula)
        angular_accel = np.linalg.inv(I) @ (M - np.cross(w, I @ w))     # (3,)   a vector [rotationx, rotationy, rotationz]
        # (3,) = (3,3) @ ((3,) - crossproduct((3,), (3,3) @ (3,)))

        # NOTE(roy) this might look a little confusing at first glance, but we're populating DERIVATIVES
        # of the state here! So position -> velocity, velocity -> acceleration, etc...
        # If you can think of a more readable way to do this (e.g. defining a QuadrotorStateDerivative class)
        # feel free to open a pull-request into the upstream repository!
        return QuadrotorState(
            position=state.velocity,
            velocity=accel,
            orientation=dRot3(state.orientation, state.angular_velocity),
            angular_velocity= angular_accel
        )


### The controller

In [43]:
@dataclass
class ControllerParams:
    """The same as above but for the controller gains"""

    # NOTE(roy) diag just means we get a matrix with these values on the diagonal,
    # so these are 3x3 matrices (print them if you're unsure!)
    # We use this formulation because it makes the math a bit more readable later on!

    # K_p is the proportional gain (spring):
    # try setting to 300 instead of 100 to get more bouncy spring action
    K_p: np.ndarray = field(default_factory=lambda: np.diag([0, 0, 80]))
                                                                        # [[  0   0   0]
                                                                        #  [  0   0   0]
                                                                        #  [  0   0 100]]
    
    # K_d is the derivative gain (damper):
    K_d: np.ndarray = field(default_factory=lambda: np.diag([0, 0, 20]))
                                                                        # [[  0   0   0]
                                                                        #  [  0   0   0]
                                                                        #  [  0   0  20]]

    # Min/max rotor rates!
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600

controller_p = ControllerParams()


In [92]:
class Controller(ControllerBase):
    @staticmethod
    def rotor_rates_from_u(u: np.ndarray) -> np.ndarray:
        """This function accepts an array u which is [thrust, Mx, My, Mz] and returns an array of required rotor rates [w1, w2, w3, w4]"""
        # NOTE(roy) implement your conversion from u (so that's [u1, u2_1, u2_2, u2_3] like we discussed, a 4-sized vector)
        # to rotor rates (which is also a four-sized vector)

        # NOTE(roy) the steps here are:
        #  1. convert u -> rotor_forces with the inverse mixing matrix
        # we are going to do the inverse of the dynamics equation, which was this:
        # u = mixing_matrix @ np.array([F1, F2, F3, F4])
        # u = desired_thrust_and_torques
        # desired_thrust_and_torques = mixing_matrix @ required_rotor_forces, which is (4,4) @ (4,)
        # so new equation will be required_rotor_forces = inverse_mixing_matrix @ desired_thrust_and_torques
        # [F1, F2, F3, F4] = inverse_mixing_matrix @ u

        L = p.arm_length
        m_t = p.k_torque / p.k_thrust     # multiply the forces by this to get the torques
        mixing_matrix = np.array([
            [  1,      1,      1,        1],
            [  0,      L,      0,      - L],
            [- L,      0,      L,        0],
            [m_t,  - m_t,    m_t,    - m_t]
        ])

        forces = np.linalg.inv(mixing_matrix) @ u       # (4,) = (3,3) @ (4,) 
        # print('forces before fixing negatives', whatami(forces))            # [-13.5853523 -13.5853523 -13.5853523 -13.5853523]
        # if any forces are NEGATIVE, set them to zero so it does not crash the square root function
        forces[0] = max(forces[0], 0)
        forces[1] = max(forces[1], 0)
        forces[2] = max(forces[2], 0)
        forces[3] = max(forces[3], 0)
        
        #  2. convert forces -> rotor rates by inverting the rotor rate model
        # F = p.k_thrust * w^2      forces =  rotor_rates * rotor_rates * p.k_thrust  (in rotor_thrust_model)
        # w^2 = forces / p.k_thrust        
        # w = square_root(forces / p.k_thrust)
        
        rotor_rates = np.sqrt(forces / p.k_thrust)        # (4,) = np.sqrt((4,) / float)

        #  3. limit the rotor rates with rotor_rate_min/rotor_rate_max above!
        rotor_rates[0] = max(rotor_rates[0], controller_p.rotor_rate_min)
        rotor_rates[1] = max(rotor_rates[1], controller_p.rotor_rate_min)
        rotor_rates[2] = max(rotor_rates[2], controller_p.rotor_rate_min)
        rotor_rates[3] = max(rotor_rates[3], controller_p.rotor_rate_min)

        rotor_rates[0] = min(rotor_rates[0], controller_p.rotor_rate_max)
        rotor_rates[1] = min(rotor_rates[1], controller_p.rotor_rate_max)
        rotor_rates[2] = min(rotor_rates[2], controller_p.rotor_rate_max)
        rotor_rates[3] = min(rotor_rates[3], controller_p.rotor_rate_max)

        # print('rotor_rates =', whatami(rotor_rates))
        return rotor_rates


    def step(self, t: float, trajectory: TrajectoryState, state: QuadrotorState) -> QuadrotorCommands:
        desired_position = trajectory.position
        desired_velocity = trajectory.velocity
        actual_position = state.position
        actual_velocity = state.velocity

        if t == 0: print('desired_position =', desired_position)
        if t == 0: print('desired_velocity =', desired_velocity)
        if t == 0: print('actual_position =', actual_position)
        if t == 0: print('actual_velocity =', actual_velocity)
        
        # in the dynamics model, the flow of information is rotor_rates -> rotor_thrusts -> body_frame_thrust_and_torques -> world_frame_thrust_and torques
        # we need to go the other way: world_frame_thrust_and_torques -> body_frame_thrust_and_torques -> rotor_thrusts -> rotor rates
        # since we assume the drone is upright, world_frame = body_frame, so we don't need to worry about the state.orientation for now
         
        # NOTE(roy) first compute the desired acceleration using the equation from the lectures!
        # I already extracted the values you need to compute it above
        # project session 5 page 17 equation is r¨= - K_d * (trajectory.velocity - state.velocity) - K_p * (trajectory.position - state.position)

        # commanded_acceleration = controller_p.K_d * (desired_velocity - actual_velocity) + controller_p.K_p * (desired_position - actual_position)
        # if t == 0: print('commanded_acceleration', whatami(commanded_acceleration))    # it turns out to be a (3,3) matrix, is this a problem?

        commanded_acceleration = controller_p.K_d @ ((desired_velocity - actual_velocity) + (desired_position - actual_position))
        if t == 0: print('commanded_acceleration', whatami(commanded_acceleration))    # 


        # NOTE(roy) now compute u_1 from the commanded acceleration using the linearized equation from the lectures!
        # project session 5 page 18 says, assuming R is at identity (drone pointing straigt up),
        # u_1 = m * (g + r¨z_des), thrust = mass times (gravity + desired z acceleration)


        # u_1 = p.mass * (p.g + commanded_acceleration[2][2])    # taking from the 3x3 matrix should be a float
        # if t == 0: print('u_1 =', u_1)

        u_1 = p.mass * (p.g + commanded_acceleration[2])    # should be a float
        if t == 0: print('u_1 =', u_1)

        # NOTE(roy) we're just doing thrust this week, set u_2 to just be zeros!
        u_2 = np.zeros(3)
        if t == 0: print('u_2 =', u_2)
        u = np.array([u_1, *u_2])
        if t == 0: print('u=', u)

        rotor_rates = Controller.rotor_rates_from_u(u)
        return QuadrotorCommands(rotor_rates)

### The trajectory
Because we're now finally doing closed-loop control, we also need to set desired states in the trajectory!
I've set up something simple, please mess around with it!

In [93]:
class JumpTrajectory(TrajectoryBase):
    def eval(self, t: float):
        # altitude = 0 if t < 1.0 else 1.0
        altitude = -.5 if t < 1.0 or t > 3.0 else .5
        # altitude = np.sin(t * np.pi) / 2
        # altitude = 3
        position = np.array([0, 0, altitude])

        return TrajectoryState(t, position=position)

### Running it!

In [94]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d, animate_matplotlib

In [95]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.01,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(),
        trajectory=JumpTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [96]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 5.0)

desired_position = [ 0.   0.  -0.5]
desired_velocity = [0. 0. 0.]
actual_position = [0. 0. 0.]
actual_velocity = [0 0 0]
commanded_acceleration is a ndarray with shape (3,) = [  0.   0. -10.]
u_1 = -0.2614092000000008
u_2 = [0. 0. 0.]
u= [-0.2614092  0.         0.         0.       ]


Output()

### Looking at the controller behavior
If all is well, the quadrotor should have a stable hover and after 1 second jump up 1 meter (with the `JumpTrajectory`)

Try some experiments:
- What happens if you increase/descrease `K_p`?
- What happens if you increase/decrease `K_d`?
- What happens if you use a slightly different mass in the controller (so the controller and dynamics have a different estimate for the quadrotor mass)? How do you think you could solve the behavior you see?

### Commit your updated notebook to your fork!